task name: Implement Prior Trainer

task description: Implement the Prior Trainer using the provided C# code and save the result to an output file.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        var label = randomFloat() > 0.3f;
        yield return new DataPoint
        {
            Label = label,
            Features = Enumerable.Repeat(label, 50)
                .Select(x => x ? randomFloat() : randomFloat() + 0.3f).ToArray()
        };
    }
}

public class DataPoint
{
    public bool Label { get; set; }
    [VectorType(50)]
    public float[] Features { get; set; }
}

var dataPoints = GenerateRandomDataPoints(1000);
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

In [ ]:
var pipeline = mlContext.BinaryClassification.Trainers.Prior();
var model = pipeline.Fit(trainingData);

In [ ]:
var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));
var transformedTestData = model.Transform(testData);

In [ ]:
public class Prediction
{
    public bool Label { get; set; }
    public bool PredictedLabel { get; set; }
}

var predictions = mlContext.Data.CreateEnumerable<Prediction>(transformedTestData, reuseRowObject: false).ToList();
foreach (var p in predictions.Take(5))
    Console.WriteLine($"Label: {p.Label}, Prediction: {p.PredictedLabel}");

In [ ]:
var metrics = mlContext.BinaryClassification.Evaluate(transformedTestData);
Console.WriteLine($"Accuracy: {metrics.Accuracy}");
Console.WriteLine($"AUC: {metrics.AreaUnderRocCurve}");
Console.WriteLine($"F1 Score: {metrics.F1Score}");

In [ ]:
using System.IO;
File.WriteAllText("output.txt", metrics.ToString());